# **Введение в функциональный API**

Функциональный API позволяет напрямую манипулировать тензорами и использовать уровни как функции, которые принимают и возвращают тензоры (чем
и обусловлено такое название — функциональный API):

In [ ]:
from keras import Input, layers

input_tensor = Input(shape=(32,))                         # Тензор
dense = layers.Dense(32, activation='relu')               # Слой — это функция
output_tensor = dense(input_tensor)                       # Вызываемый слой может принимать и возвращать тензор

In [1]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Уже знакомая нам модель Sequential
seq_model = Sequential() 
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Ее функциональный эквивалент
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)                    #Класс Model превращает входной и выходной тензоры в модель

model.summary()     # Рассмотрим ее!


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

import numpy as np                                      # Генерация фиктивных данных для обучения
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train) 

Epoch 1/10
8/8 [==============================] - 1s 3ms/step - loss: 11.6928
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 12.0221
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 12.8207
Epoch 4/10
8/8 [==============================] - 0s 3ms/step - loss: 14.2873
Epoch 5/10
8/8 [==============================] - 0s 3ms/step - loss: 16.6177
Epoch 6/10
8/8 [==============================] - 0s 3ms/step - loss: 19.9864
Epoch 7/10
8/8 [==============================] - 0s 4ms/step - loss: 23.7409
Epoch 8/10
8/8 [==============================] - 0s 4ms/step - loss: 27.1165
Epoch 9/10
8/8 [==============================] - 0s 4ms/step - loss: 30.7547
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 36.8690


# **Модели с несколькими входами**

In [1]:
# Реализация модели «вопрос/ответ» с двумя входами с использованием функционального API

from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')           # Входной текст — это последовательность целых чисел переменной
                                                                         # длины. Обратите внимание на то, что при желании можно задать имя
                                                                         # последовательности

embedded_text = layers.Embedding(                                       # Преобразование входного текста в последовательность векторов с размером 64
          text_vocabulary_size, 64)(text_input)         

encoded_text = layers.LSTM(32)(embedded_text)                           # Преобразование векторов в единый вектор с помощью уровня LSTM

question_input = Input(shape=(None,),                                   # Та же процедура (с другими экземплярами слоев) повторяется для вопроса
                      dtype='int32',
                      name='question')

embedded_question = layers.Embedding(                                   
                    question_vocabulary_size, 32)(question_input)

encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question],     # Объединение закодированных вопроса и текста
                                  axis=-1) 

answer = layers.Dense(answer_vocabulary_size,                           # Добавление классификатора softmax сверху
                      activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)                     # Создание экземпляра модели с двумя входами и одним выходом

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
# Передача данных в модель с несколькими входами

import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,                             # Создание массива Numpy с фиктивными данными
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,                     # К вопросам применяется прямое кодирование, 
                            size=(num_samples, max_length))                    # а не преобразование в целые числа
                            
answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1

model.fit([text, question], answers, epochs=10, batch_size=128)               # Передача списка входов

model.fit({'text': text, 'question': question}, answers,                      # Передача с помощью словаря (возможна, только если были определены имена для входов)
          epochs=10, batch_size=128) 

# **Модели с несколькими выходами**

In [ ]:
#  Реализация модели с тремя выходами с использованием функционального API

from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)                                 # Обратите внимание: для выходных слоев определены имена
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

In [ ]:
# Параметры компиляции модели с несколькими выходами: несколько функций потерь

model.compile(optimizer='rmsprop',
 loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop',                                  # Эквивалентное решение (возможно, только если
              loss={'age': 'mse',                                    # определены имена выходных слоев)
              'income': 'categorical_crossentropy',
              'gender': 'binary_crossentropy'}) 

In [ ]:
# Параметры компиляции модели с несколькими выходами: взвешивание потерь

model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',                                       # Эквивалентное решение (возможно, только если
              loss={'age': 'mse',                                         # определены имена выходных слоев)
              'income': 'categorical_crossentropy',
              'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
              'income': 1.,
              'gender': 10.})

In [ ]:
# Передача данных в модель с несколькими выходами

model.fit(posts, [age_targets, income_targets, gender_targets], # Предполагается, что age_targets, income_targets и gender_targets — это массивы Numpy
          epochs=10, batch_size=64)

model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
                  epochs=10, batch_size=64) 

# **Ориентированные ациклические графы уровней**

## Модули Inception

In [ ]:
# Модуль Inception
# Этот пример предполагает наличие четырехмерного входного тензора x:

from keras import layers

# Все ветви имеют одинаковый шаг свертки (2). Это необходимо для получения на выходе всех ветвей тензоров одного размера,
 # чтобы потом их можно было объединить
branch_a = layers.Conv2D(128, 1,
                        activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b) # В этой ветви шагание происходит в слое пространственной свертки


branch_c = layers.AveragePooling2D(3, strides=2)(x)                      # В этой ветви шагание происходит в слое извлечения среднего по соседям
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate(                                              # Объединение результатов, возвращаемых ветвями для получения результата работы модуля
          [branch_a, branch_b, branch_c, branch_d], axis=-1)

# Доступно в keras.applications .inception_v3.InceptionV3
# Там же есть Xception, быстрее и качественее

## Остаточные связи

Остаточная связь заключается в передаче вывода более раннего слоя на вход более
позднего слоя, вследствие чего в последовательной сети фактически создается
короткий путь. Вместо объединения с более поздней активацией вывод, полученный ранее, суммируется с более поздней активацией, что предполагает равенство
размеров обеих активаций. Если они имеют разные размеры, можно применить
линейное преобразование для приведения формы ранней активации к форме цели
(например, слой Dense без активации или, для карт сверточных признаков, свертку 1 × 1 без активации).

Вот как можно реализовать остаточную связь в Keras, когда размеры карт признаков совпадают. Этот пример предполагает наличие четырехмерного входного
тензора x:

In [ ]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x) # Применение преобразования к x
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])                                          # Добавление оригинального тензора x к выходным признакам

А вот так реализуется остаточная связь, когда размеры карт признаков различаются
(и снова предполагается наличие четырехмерного входного тензора x):

In [ ]:
from keras import layers

x = ...
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)    # Используется свертка 1 × 1 для линейного
                                                                   # снижения размерности исходного тензора x,
                                                                   # чтобы получить форму как у тензора y

y = layers.add([y, residual])                                     # Добавление остаточного тензора к выходным признакам


# **Повторное использование экземпляров слоев**

Еще одной важной особенностью функционального API является возможность повторного использования экземпляра слоя. Когда вы дважды вызываете экземпляр
слоя, вместо создания нового слоя в каждом вызове повторно будут использоваться
те же самые веса. Это позволяет создавать модели с общими ветвями, когда имеется
несколько ветвей, совместно использующих общие знания и выполняющих одинаковые операции. Другими словами, они вместе используют общие представления
и совместно обучают их на разных входных наборах.

Например, рассмотрим модель, которая попытается оценить семантическое сходство двух предложений. Модель имеет два входа (два сравниваемых предложения)
и выводит оценку в диапазоне между 0 и 1, где 0 означает полное отсутствие сходства между предложениями, а 1 — полную смысловую идентичность. Такая модель
могла бы найти массу применений, включая устранение избыточных запросов на
естественном языке в диалоговых системах.

В такой конфигурации два входных предложения взаимозаменяемы, потому что
семантическое сходство является симметричным отношением: сходство А с Б идентично сходству Б с А. По этой причине нецелесообразно обучать две независимые
модели для обработки каждого входного предложения. Предпочтительнее было
бы обрабатывать оба одним слоем LSTM. Представления этого слоя LSTM (его веса)
определяются на основе обоих входов одновременно. Мы называем это сиамской
моделью LSTM, или общим LSTM.

Вот как можно реализовать такую модель с использованием приема совместного (или повторного) использования слоя в функциональном API фреймворка
Keras:

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)                                            # Создание единственного экземпляра слоя LSTM, выполняется однократно

left_input = Input(shape=(None, 128))                             # Конструирование левой ветви модели: на вход подаются
left_output = lstm(left_input)                                     # последовательности переменной длины векторов с размерностью 128

right_input = Input(shape=(None, 128))                            # Конструирование правой ветви модели: обращаясь к существующему
right_output = lstm(right_input)                                   # экземпляру слоя, вы повторно используете его веса

merged = layers.concatenate([left_output, right_output], axis=-1) # Добавление классификатора сверху
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)             # Создание и обучение модели: в процессе обучения такой модели веса слоя LSTM
                                                                   # обновляются в процессе обработки обоих входов
model.fit([left_data, right_data], targets)

# **Модели как слои**

Простым примером практического применения повторного использования экземпляра модели может служить модель зрения, которая в качестве входа использует
сдвоенную камеру: две параллельные камеры, отстоящие друг от друга на пару
сантиметров (один дюйм). Такая модель может воспринимать глубину, что может
пригодиться во многих приложениях. Вам не нужно создавать две независимые
модели для извлечения визуальных признаков из изображений, передаваемых
левой и правой камерами, перед объединением двух потоков. Низкоуровневую
обработку двух входных потоков можно выполнять сообща, то есть задействовать
слои, совместно использующие одни и те же веса и, соответственно, представления. Вот как можно реализовать сиамскую модель зрения (с общей сверточной
основой) в Keras:

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weights=None,           # Базовая модель обработки изображения — сеть Xception (только сверточная основа)
                                      include_top=False)

left_input = Input(shape=(250, 250, 3))                       # На вход подаются изображения в формате RGB и с размером 250 × 250
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_input = xception_base(right_input)                      # Одна и та же модель вызывается дважды

merged_features = layers.concatenate(                         # Объединенный набор признаков содержит информацию из правого
                     [left_features, right_input], axis=-1)    # и левого источников визуальной информации
                 